# Loading data from an existing Experiment Pipeline

In [ ]:
import pandas as pd
from azureml.core import Workspace
ws = Workspace.from_config()

In [ ]:
from azureml.core import Experiment
experiment = Experiment(ws, 'longterm')

In [ ]:
experiment

In [ ]:
pipeline_run = None
for run in experiment.get_runs():
    if run.get_status()=='Finished':
        pipeline_run = run

In [ ]:
run

In [ ]:
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineData

# Default datastore
default_store = ws.get_default_datastore() 

data_with_id = PipelineData("data_with_id", datastore=default_store).as_dataset()

transformStep = PythonScriptStep(
    name="Add trip_id",
    script_name="trip_id.py", 
   
)

In [ ]:
# functions to download output to local and fetch as dataframe
def get_download_path(download_path, output_name):
    output_folder = os.listdir(download_path + '/azureml')[0]
    path =  download_path + '/azureml/' + output_folder + '/' + output_name
    return path

def fetch_df(step, output_name):
    output_data = step.get_output_data(output_name)    
    download_path = './outputs/' + output_name
    output_data.download(download_path, overwrite=True)
    df_path = get_download_path(download_path, output_name) + '/processed.parquet'
    return pd.read_parquet(df_path)

In [ ]:
trip_id_step = pipeline_run.find_step_run("Add trip_id")[0]
data_with_id = fetch_df(trip_id_step, 'data_with_id')

display(data_with_id.describe())